In [15]:
import pandas as pd
from time import sleep
from slugify import slugify
import random
import datetime

from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

from selenium_stealth import stealth

pd.set_option('display.max_colwidth', None)
%matplotlib inline

In [7]:
# nazi luftwaffe
# nazi kriegsmarine
# nazi heer
# Schutzstaffel
# Third Reich
Nazi SS

NameError: name 'Luftwaffe' is not defined

In [16]:
def generateDriver():
    options = webdriver.ChromeOptions() 
    options.add_argument("user-data-dir=selenium")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage') 
    options.add_argument("start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled") 
    options.add_experimental_option('excludeSwitches', ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(options = options)
    stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine")

    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})") 
    driver.get('https://www.liveauctioneers.com/search/')
    return driver

driver = generateDriver()

In [ ]:
urls = []
y = 1999
term = 'nazi ss'
escapedTerm = '%20'.join(term.split(' '))

while y < 2024:
    start = int(datetime.datetime.strptime('01/01/' + str(y), "%d/%m/%Y").timestamp())
    end = int(datetime.datetime.strptime('31/12/' + str(y) + ' 23:59:59', "%d/%m/%Y %H:%M:%S").timestamp())
    time = '&saleDateMax=' + str(end) + '&saleDateMin=' + str(start)
    search = 'https://www.liveauctioneers.com/search/?country=US&keyword=' + escapedTerm + time + '&sort=-saleStart&status=archive'
    driver.get(search)
    if (driver.find_elements(By.CSS_SELECTOR, '[class*="SearchResults__StyledResults"] div:last-child div:last-child [data-testid="select"]')):
        driver.find_element(By.CSS_SELECTOR, '[class*="SearchResults__StyledResults"] div:last-child div:last-child [data-testid="select"]').click()
        sleep(2)
        if (driver.find_elements(By.CSS_SELECTOR, '#react-select-pagesize-option-4')):
            driver.find_element(By.CSS_SELECTOR, '#react-select-pagesize-option-4').click()
    while True:
        sleep(random.randint(15,25))
        try:
            listings = driver.find_elements(By.CSS_SELECTOR, 'article[class*="CategorySearchCard__StyledCard"]')
            for c in listings:
                url = c.find_element(By.CSS_SELECTOR, '[data-testid="itemCard"]').get_attribute('href')    
                urls.append(url)         
        except (exceptions.NoSuchElementException, exceptions.StaleElementReferenceException):
            break
        if (driver.find_elements(By.CSS_SELECTOR, 'li.next.disabled') or len(listings) == 0):
            break
        more = driver.find_element(By.CSS_SELECTOR, '[class*="SearchResults__StyledResults"] div:last-child ul li:last-child a')   
        more.click()
    print(len(urls), driver.current_url)
    y = y + 1
    
print('Done!', len(list(set(urls))))

0 https://www.liveauctioneers.com/search/?country=US&keyword=nazi%20ss&saleDateMax=946709999&saleDateMin=915174000&sort=-saleStart&status=archive
0 https://www.liveauctioneers.com/search/?country=US&keyword=nazi%20ss&saleDateMax=978332399&saleDateMin=946710000&sort=-saleStart&status=archive
0 https://www.liveauctioneers.com/search/?country=US&keyword=nazi%20ss&saleDateMax=1009868399&saleDateMin=978332400&sort=-saleStart&status=archive
0 https://www.liveauctioneers.com/search/?country=US&keyword=nazi%20ss&saleDateMax=1041404399&saleDateMin=1009868400&sort=-saleStart&status=archive
2 https://www.liveauctioneers.com/search/?country=US&keyword=nazi%20ss&pageSize=120&saleDateMax=1072940399&saleDateMin=1041404400&sort=-saleStart&status=archive
2 https://www.liveauctioneers.com/search/?country=US&keyword=nazi%20ss&saleDateMax=1104562799&saleDateMin=1072940400&sort=-saleStart&status=archive
3 https://www.liveauctioneers.com/search/?country=US&keyword=nazi%20ss&pageSize=120&saleDateMax=11360987

In [22]:
df_urls = pd.DataFrame(list(set(urls)), columns=["url"])
df_urls.to_csv('data/urls_' + '_'.join(term.split(' ')) + '.csv', index = False)

In [40]:
all_files = [
    'data/urls_nazi_heer.csv',
    'data/urls_nazi_kriegsmarine.csv',
    'data/urls_nazi_luftwaffe.csv',
    'data/urls_nazi_ss.csv',
    'data/urls_schutzstaffel.csv',
    'data/urls_third_reich.csv',
    'data/urls_wehrmacht.csv'
]
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True).drop_duplicates()
df1 = pd.read_csv('data/urls_nazi_swastika.csv')
l3 = [x for x in df.url.tolist() if x not in df1.url.tolist()]
pd.DataFrame(l3, columns=["url"]).to_csv('data/urls_not_swastika.csv', index = False)